<center><h1>Hands-On Reinforcement Learning Applied to Trade Execution Algorithms</h1></center>
<center>
Autor: <cite><a href="https://www.linkedin.com/in/aoteog/">Oteo García, Alberto</a></cite>
</center>
<center>
Autor: <cite><a href="https://www.linkedin.com/in/jesus-sanz/">Sanz del Real, Jesús</a></cite>
</center>

----------------


In [ ]:
import pickle
from typing import Any, List, Sequence, Tuple
import sys

import numpy as np
import pandas as pd

sys.path.append("../src/")

In [ ]:
with open("../data/orderbook.pkl", "rb") as f:
        dict_ = pickle.load(f)
dict_.keys()

In [ ]:
data = dict_['train']
look_back = 60

In [ ]:
"""Inicialización de la clase del entorno que simula
el libro de ordenes.
----------------------------------------------------
Input:
    - data: 
        Dataframe con los datos previamente
        agrupados del libro de órdenes.

    - look_back: 
        Ventana para la generación de features
        roladas en el instante t=0 del episodio.
        Esta ventana representa el rango máximo para
        la construcción de features.
----------------------------------------------------
Variables Internas:
    - episode_bins:
        Número de bines (steps) del episodio.
    - episode_full_len:
        Es igual a look_back + episode_bins.
    - vol_care:
        Volumen total (en títulos) de la orden care.
    - actions_fn:
        Diccionario con las posibles acciones del agente.
        Las claves acceden a la función que evalúa  la acción
        tomada por el agente.
    - n_actions:
        Número de acciones posibles.
    - n_features:
        Número de características de los estados.
    - episode:
        Dataframe que contiene los steps y estados del episodio.
    - episode_full:
        Es el episode añadiendo el look_back antes del comienzo 
        del episodio.
    - episode_vwap:
        VWAP de mercado al final del episodio.
    - market_ep_vol:
        Volumen (títulos) ejecutado por el mercado en cada bin del episodio.
    - state_pos:
        Número de step en el que nos encontramos.
    - exec_vol:
        Acumulado de títulos ejecutados por el algoritmo.
    - action_hist:
        Lista de acciones tomadas por el algoritmo en cada step.
    - market_vwap_hist:
        Lista de VWAP de mercado en cada step.
    - reward_hist:
        Lista de rewards obtenidas en cada step.
    - price_hist:
        Lista de precios ejecutados en cada step.
    - vol_hist:
        Lista de títulos ejecutados en cada step.                
"""

# Fixed params
data = data
look_back = look_back
episode_bins = None
episode_full_len = None
vol_care = None

def _do_nothing():
    return

def _agg_action():
    return

actions_fn = {
    0: _do_nothing,
    1: _agg_action,
}

n_actions = len(actions_fn)

def _detect_num_feat():
    return

n_features = _detect_num_feat()

# Data variables
episode = None
episode_full = None

# Env variables
episode_vwap = None
market_ep_vol = None
state_pos = 0
exec_vol = 0
actions_hist = []
algo_vwap_hist = []
market_vwap_hist = []
reward_hist = []
price_hist = []
vol_hist = []

### 1. _generate_episode_params

In [ ]:
# def _generate_episode_params():
"""Se determinan las características de la orden a ejecutar.
La órden queda definida por: 
 - episode_bins:
     Obtención de un número entero aleatorio [400, 600] 
     con una distribución uniforme.
 - vol_care:
     Obtención del porcentaje de steps en el que hay que 
     ejecutar una órden para cubrir la órden care. 
     vol_care responde a un valor uniforme [0.075, 0.125]
     multiplicado por el número self.episode_bins. 
     Lo convertimos a entero.
"""
# TODO: Int aleatorio entre 400 y 600 como un objeto numpy
episode_bins = np.random.randint(low=400, high=600)
# TODO: Float aleatorio entre 0.075 y 0.125
pct_bins = np.random.uniform(low=0.075, high=0.125)
# TODO: Int multiplicacion pct_bins y episode_bins
vol_care = int(pct_bins * episode_bins)

episode_full_len = episode_bins + look_back

assert episode_bins <= 600
assert episode_bins >= 400
assert vol_care <= int(episode_bins * 0.125)
assert vol_care >= int(episode_bins * 0.075)
assert isinstance(vol_care, int)

def _generate_episode_params():
    episode_bins = np.random.randint(low=400, high=600)
    pct_bins = np.random.uniform(low=0.075, high=0.125)
    vol_care = int(pct_bins * episode_bins)
    episode_full_len = episode_bins + look_back

### 2. _generate_episode

In [ ]:
# def _generate_episode():
"""Obtenemos el día y hora en el que comienza el episodio.
"""
lenght_episode = 0
while lenght_episode != episode_full_len:
    # TODO: Selección de un dia entre los posibles.
    # Clue: Usa np.random.choice y los dias data.keys
    selected_day = np.random.choice(
            list(data.keys()
        )
    )

    # TODO: Extrae selected_day de data
    data_day = data[selected_day]

    # TODO: selecciona una hora de inicio aleatoria
    init_time = np.random.choice(data_day.index)

    hour_pos = data_day.index.get_loc(init_time)
    initial_position = hour_pos - look_back
    final_position = hour_pos + episode_bins

    if initial_position < 0:
        continue
    else:
        # TODO: Filtra data_day entre por initial_position y final_position
        episode_full = data_day.iloc[initial_position:final_position, :]

        # TODO: Filtra data_day entre por hour_pos y final_position
        episode = data_day.iloc[hour_pos:final_position, :]

        lenght_episode = episode_full.shape[0]

def _generate_episode():
    lenght_episode = 0
    while lenght_episode != episode_full_len:
        selected_day = np.random.choice(
                list(data.keys()
            )
        )
        data_day = data[selected_day]
        init_time = np.random.choice(data_day.index)
        hour_pos = data_day.index.get_loc(init_time)
        initial_position = hour_pos - look_back
        final_position = hour_pos + episode_bins
        if initial_position < 0:
            continue
        else:
            episode_full = data_day.iloc[initial_position:final_position, :]
            episode = data_day.iloc[hour_pos:final_position, :]
            lenght_episode = episode_full.shape[0]

### 3. observation_builder

In [ ]:
# def observation_builder() -> np.array:
""" Función para la construcción de las observaciones del estado.
    ------------------------------------------------------------
    Default:
        - Primera característica es tiempo restante en porcentaje.
        - Seguna característica es el volumen restante en porcentaje.
"""
# TODO: Construye el vector con las dos características de la descripción
# Clue: Utiliza episode_bins, state_pos, exec_vol ,vol_care
time_left = (episode_bins - state_pos) / episode_bins
vol_left = 1 - (exec_vol / vol_care)
obs  = np.array([time_left, vol_left])

print(obs)

def observation_builder() -> np.array:
    time_left = (episode_bins - state_pos) / episode_bins
    vol_left = 1 - (exec_vol / vol_care)
    obs  = np.array([time_left, vol_left])
    return obs

### 4. _compute_episode_market_feat

In [ ]:
# def _compute_episode_market_feat() -> Tuple[float, float]:
"""Cálculo de los valores VWAP y Market Vol del episodio.
Como no tenemos las ejecuciones de mercado, asumimos que el 
precio es el mid price de cada step.
"""
# TODO: Calcula el mid price utilizando ask1 y bid1 de episode
# Opcional: Utiliza un precio más realista para el mkt VWAP
mid = (episode["ask1"] + episode["bid1"]) / 2
# TODO: Calcula market_ep_vol
market_ep_vol = episode.cumvol.diff()
market_ep_vol[0] = 0
# TODO: calcula el volumen acumulado del mercado en todo el episodio
cum_vol = market_ep_vol.sum()
# TODO: calcula el episode_vwap
episode_vwap = (mid[:-1] * market_ep_vol[1:]).sum() / cum_vol

print(episode_vwap)
print(market_ep_vol)

def _compute_episode_market_feat(self) -> Tuple[float, float]:
    mid = (episode["ask1"] + episode["bid1"]) / 2
    # TODO: Calcula market_ep_vol
    market_ep_vol = episode.cumvol.diff()
    market_ep_vol[0] = 0
    # TODO: calcula el volumen acumulado del mercado en todo el episodio
    cum_vol = market_ep_vol.sum()
    # TODO: calcula el episode_vwap
    episode_vwap = (mid[:-1] * market_ep_vol[1:]).sum() / cum_vol
    return episode_vwap, market_ep_vol

### 5. _compute_market_vwap

In [ ]:
# def _compute_market_vwap() -> float:
"""Cálculo del VWAP del mercado hasta el step actual.
"""
# TODO: Establece un para el vol ejecutado por el mkt en cada step
# Clue: puedes fijarte en _compute_episode_market_feat
mid_p = (episode["ask1"] + episode["bid1"]) / 2
mkt_p = (mid_p + mid_p.shift(-1).ffill()) / 2
# Calcula todos los vwap del mkt hasta el step actual incluido
v = episode["cumvol"].diff().shift(-1)
p_arr = mkt_p.values[:state_pos + 1]
v_arr = v.values[:state_pos + 1]
sum_vol = np.sum(v_arr)
# Si el mkt vol hasta el step == 0, devuelve el último precio hasta el step
if sum_vol == 0:
    market_vwap = p_arr[-1]
else:
    # Calcula y devuelve el vwap acumulado hasta el step
    market_vwap = np.sum(p_arr * v_arr) / sum_vol

print(market_vwap)

def _compute_market_vwap() -> float:
    mid_p = (episode["ask1"] + episode["bid1"]) / 2
    mkt_p = (mid_p + mid_p.shift(-1).ffill()) / 2
    v = episode["cumvol"].diff().shift(-1)
    p_arr = mkt_p.values[:state_pos + 1]
    v_arr = v.values[:state_pos + 1]
    sum_vol = np.sum(v_arr)
    if sum_vol == 0:
        return p_arr[-1]
    market_vwap = np.sum(p_arr * v_arr) / sum_vol
    return market_vwap

### 6. _compute_reward

In [ ]:
price = 0
vol = 0

In [ ]:
# def _compute_reward(price: float, vol: float) -> float:
"""Función de diseño de los rewards y penalizaciónes que 
recibe el algoritmo al tomar las acciones.
--------------------------------------------------------
Default:
    - El reward es el ratio de la diferencia entre el episode_vwap y
      el precio de la acción tomada, dividido entre episode_vwap.
"""
# TODO: Establece y devuelve un reward cuando vol == 0
if vol == 0:
    reward = 0
    print(reward)
# TODO: Calcula y devuelve el reward cuando vol > 0
# Clue: Utiliza episode_vwap y price para la reward por defecto
# Opcional: Utiliza el self y elimina los parámetros de la función
reward = (episode_vwap - price) / episode_vwap
print(reward)

def _compute_reward(price: float, vol: float) -> float:
    if vol == 0:
        reward = 0
        return reward
    reward = (episode_vwap - price) / episode_vwap
    return reward

### 7-8. _agg_action & _compute_algo_vwap

In [ ]:
# def _compute_algo_vwap() -> float:
"""Cálculo del VWAP del algoritmo hasta el step actual.
"""
# TODO: Calcula el algo_vwap
# Clue: utiliza price_hist, vol_hist
p_arr = np.array(price_hist)
v_arr = np.array(vol_hist)
algo_vwap = np.sum(p_arr * v_arr) / np.sum(v_arr)

print(algo_vwap)

def _compute_algo_vwap() -> float:
    p_arr = np.array(price_hist)
    v_arr = np.array(vol_hist)
    algo_vwap = np.sum(p_arr * v_arr) / np.sum(v_arr)
    return algo_vwap

In [ ]:
# def _agg_action() -> float:
"""Acción agresiva de compra de un título a precio de episode['ask1'].
Devolvemos el reward asociado a esa acción.
"""
# TODO: obtén el precio de la accion agresiva (ask1) en el state_pos
price = episode["ask1"].values[state_pos]
# TODO: guarda price en price_hist, añade 1 a exec_vol y añade 1 a vol_hist
price_hist.append(price)
exec_vol = 1
exec_vol += exec_vol
vol_hist.append(exec_vol)

# TODO: utiliza la función apropiada para calcula el algo_vwap
algo_vwap = _compute_algo_vwap()
# guarda el algo_vwap en algo_vwap_hist
algo_vwap_hist.append(algo_vwap)
# TODO: calcula el reward utilizando la función apropiada
reward = _compute_reward(price, exec_vol)
print(reward)

def _agg_action() -> float:
    price = episode["ask1"].values[state_pos]
    price_hist.append(price)
    exec_vol = 1
    exec_vol += exec_vol
    vol_hist.append(exec_vol)
    algo_vwap = _compute_algo_vwap()
    algo_vwap_hist.append(algo_vwap)
    reward = _compute_reward(price, exec_vol)
    return reward

### 9. _do_nothing

In [ ]:
# def _do_nothing() -> float:
"""No hacer nada y devolvemos el reward asociado a la acción
"""
# TODO: Repite el proceso de _agg_action
# Clue: Precio y volumen ejecutado = 0
price = 0
exec_vol = 0
price_hist.append(price)
vol_hist.append(exec_vol)
algo_vwap = algo_vwap_hist[-1]
algo_vwap_hist.append(algo_vwap)
reward = _compute_reward(price, exec_vol)

print(reward)

def _do_nothing() -> float:
    price = 0
    exec_vol = 0
    price_hist.append(price)
    vol_hist.append(exec_vol)
    algo_vwap = algo_vwap_hist[-1]
    algo_vwap_hist.append(algo_vwap)
    reward = _compute_reward(price, exec_vol)
    return reward

### 10. _compute_stop_conditions

In [ ]:
# def _compute_stop_conditions(self) -> Tuple[bool, bool]:
"""Define las condiciones de parada del episodio
Return:
    Tiempo agotado, orden completada
"""
# TODO: Calcula las variables de parada y devuélvelas en el orden apropiado
is_bins_complete = state_pos == episode_bins
is_ord_complete = exec_vol == vol_care

print(is_bins_complete)
print(is_ord_complete)

def _compute_stop_conditions() -> Tuple[bool, bool]:
    is_bins_complete = state_pos == episode_bins
    is_ord_complete = exec_vol == vol_care
    return is_bins_complete, is_ord_complete

### 11. _compute_done

In [ ]:
# def _compute_done(self) -> bool:
""" Reglas de finalización del episodio.
"""
# TODO: Calcula las condiciones de parada utilizando la función adecuada
conditions = _compute_stop_conditions()
is_bins_complete = conditions[0]
is_ord_complete = conditions[1]
# TODO: Devuelve done == True si se cumplen cualquiera de las condiciones
done = is_bins_complete or is_ord_complete

print(done)

def _compute_done() -> bool:
    conditions = _compute_stop_conditions()
    is_bins_complete = conditions[0]
    is_ord_complete = conditions[1]
    done = is_bins_complete or is_ord_complete
    return done

### 12. _compute_done_reward

In [ ]:
# def _compute_done_reward(self) -> float:
# TODO: Free style
_, is_ord_complete = _compute_stop_conditions()
rwd_factor = not is_ord_complete
done_reward = -1 * rwd_factor

print(done_reward)

def _compute_done_reward() -> float:
    _, is_ord_complete = _compute_stop_conditions()
    rwd_factor = not is_ord_complete
    done_reward = -1 * rwd_factor
    return done_reward

### 13. action_sample

In [ ]:
# def action_sample() -> int:
"""
Devuelve una acción aleatoria. El valor ha de corresponder 
con las keys de actions_fn.
"""
# TODO: Toma una acción aleatoria
# Opcional: ¿Qué distribución de prob es mejor para la exploración?
p = vol_care / episode.shape[0]
action = np.random.choice([0, 1], p=[1-p, p])

print(action)

def action_sample() -> int:
    p = vol_care / episode.shape[0]
    action = np.random.choice([0, 1], p=[1-p, p])
    return action

_________

### step
Comprobemos que todo va bien

In [ ]:
action = action_sample()

In [ ]:
# def step(action) -> Tuple[np.array, float, bool, dict]:
""" Evalua la acción, calcula la recompensa, devuelve el 
nuevo estado y si el episodio ha terminado.
"""
market_vwap = _compute_market_vwap()
act_fn = actions_fn.get(action)
if act_fn is None:
    raise ValueError(
        f"Invalid action {action}. Valid actions {actions_fn.keys()}"
    )

reward = act_fn()

market_vwap_hist.append(market_vwap)
reward_hist.append(reward)

state_pos += 1

done = _compute_done()

if done:
    reward += _compute_done_reward()
    observation = None
else:
    observation = observation_builder()

print(f'observation: {np.array(observation)}')
print(f'reward: {reward}')
print(f'done: {done}')